<center>
<img src="../../img/ods_stickers.jpg">
## Отворен курс по машинно обучение
</center>
Автор на материала: програмист-изследовател в Mail.ru Group, старши преподавател във Факултета по компютърни науки на Висшето училище по икономика Юрий Кашницки. Изработено на базата на курса "Изграждане на изводи от данни" на специализацията "Машинно обучение и анализ на данни" на Yandex и MIPT

# <center>Тема 9. Анализ на времеви редове в Python</center>
## <center>Част 2. Смъртни случаи при злополука в Съединените щати</center>

Известен е месечният брой на смъртните случаи поради злополуки в Съединените щати от януари 1973 г. до декември 1978 г., необходимо е да се изгради прогноза за следващите 2 години.

In [ ]:
import warnings

warnings.filterwarnings("ignore")

%matplotlib inline
from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = 12, 10
from itertools import product

import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
from scipy import stats


def invboxcox(y, lmbda):
    if lmbda == 0:
        return np.exp(y)
    else:
        return np.exp(np.log(lmbda * y + 1) / lmbda)

In [ ]:
deaths = pd.read_csv(
    "../../data/accidental-deaths-in-usa-monthly.csv",
    index_col=["Month"],
    parse_dates=["Month"],
)
deaths.rename(
    columns={"Accidental deaths in USA: monthly, 1973 ? 1978": "num_deaths"},
    inplace=True,
)
deaths["num_deaths"].plot()
plt.ylabel("Accidental deaths");

Проверка на стационарност и STL разлагане на серия:

In [ ]:
sm.tsa.seasonal_decompose(deaths["num_deaths"]).plot()
print(
    "Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(deaths["num_deaths"])[1]
)

### Стационарност

Критерият на Дики-Фулър не отхвърля хипотезата за нестационарност, но остава малка тенденция. Нека опитаме сезонна диференциация; Нека направим STL декомпозиция върху диференцираните серии и проверим стационарността:


In [ ]:
deaths["num_deaths_diff"] = deaths["num_deaths"] - deaths["num_deaths"].shift(12)
sm.tsa.seasonal_decompose(deaths["num_deaths_diff"][12:]).plot()
print(
    "Тест на Дики-Фулър: p=%f"
    % sm.tsa.stattools.adfuller(deaths["num_deaths_diff"][12:])[1]
)

Критерият на Дики-Фулър отхвърля хипотезата за нестационарност, но не беше възможно напълно да се отървем от тенденцията. Нека се опитаме да добавим малко по-обикновено разграничение:

In [1]:
deaths["num_deaths_diff2"] = deaths["num_deaths_diff"] - deaths[
    "num_deaths_diff"
].shift(1)
sm.tsa.seasonal_decompose(deaths["num_deaths_diff2"][13:]).plot()
print(
    "Тест на Дики-Фулър: p=%f"
    % sm.tsa.stattools.adfuller(deaths["num_deaths_diff2"][13:])[1]
)

NameError: name 'deaths' is not defined

Хипотезата за нестационарност е уверено отхвърлена и визуално серията изглежда по-добре - вече няма тенденция.

## Избор на модел

Нека да разгледаме ACF и PACF на получената серия:

In [ ]:
ax = plt.subplot(211)
sm.graphics.tsa.plot_acf(
    deaths["num_deaths_diff2"][13:].values.squeeze(), lags=58, ax=ax
)
ax = plt.subplot(212)
sm.graphics.tsa.plot_pacf(
    deaths["num_deaths_diff2"][13:].values.squeeze(), lags=58, ax=ax
);

Първоначални приближения: Q=2, q=1, P=2, p=2

In [ ]:
ps = range(0, 3)
d = 1
qs = range(0, 1)
Ps = range(0, 3)
D = 1
Qs = range(0, 3)

In [ ]:
parameters = product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

In [ ]:
%%time
results = []
best_aic = float("inf")


for param in parameters_list:
# опит освен е необходим, защото моделът не е обучен на някои набори от параметри
    try:
        model = sm.tsa.statespace.SARIMAX(
            deaths["num_deaths"],
            order=(param[0], d, param[1]),
            seasonal_order=(param[2], D, param[3], 12),
        ).fit(disp=-1)
# показване на параметрите, на които моделът не е обучен и преминаване към следващия набор
    except ValueError:
        print("wrong parameters:", param)
        continue
    aic = model.aic
# запазете най-добрия модел, aic, параметри
    if aic < best_aic:
        best_model = model
        best_aic = aic
        best_param = param
    results.append([param, model.aic])

warnings.filterwarnings("default")

Ако възникне грешка в предишната клетка, не забравяйте да актуализирате statsmodels поне до версия 0.8.0rc1.

In [ ]:
result_table = pd.DataFrame(results)
result_table.columns = ["parameters", "aic"]
print(result_table.sort_values(by="aic", ascending=True).head())

Най-добър модел:


In [ ]:
print(best_model.summary())

Неговите останки:

In [ ]:
plt.subplot(211)
best_model.resid[13:].plot()
plt.ylabel(u"Residuals")

ax = plt.subplot(212)
sm.graphics.tsa.plot_acf(best_model.resid[13:].values.squeeze(), lags=48, ax=ax)

print("Критерий Стьюдента: p=%f" % stats.ttest_1samp(best_model.resid[13:], 0)[1])
print(
    "Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(best_model.resid[13:])[1]
)

Остатъчните стойности са безпристрастни (потвърдени от теста на Стюдънт), стационарни (потвърдени от теста на Дики-Фулър и визуално) и неавтокорелирани (потвърдени от теста на Люнг-Бокс и корелограмата).
Нека да видим колко добре моделът описва данните:

In [ ]:
deaths["model"] = best_model.fittedvalues
deaths["num_deaths"].plot()
deaths["model"][13:].plot(color="r")
plt.ylabel("Accidental deaths");

### Прогноза

In [ ]:
from dateutil.relativedelta import relativedelta

deaths2 = deaths[["num_deaths"]]
date_list = [
    pd.datetime.strptime("1979-01-01", "%Y-%m-%d") + relativedelta(months=x)
    for x in range(0, 24)
]
future = pd.DataFrame(index=date_list, columns=deaths2.columns)
deaths2 = pd.concat([deaths2, future])
deaths2["forecast"] = best_model.predict(start=72, end=100)

deaths2["num_deaths"].plot(color="b")
deaths2["forecast"].plot(color="r")
plt.ylabel("Accidental deaths");